1. Carga de datasets

In [ ]:
import pandas as pd
import numpy as np

# Cargar dataset Alemania
df_de_raw = pd.read_csv('df_de.csv', sep=',', encoding='latin-1', on_bad_lines='skip')

# Cargar dataset España - CORRECCIÓN: separador es COMA, no punto y coma
df_es_raw = pd.read_csv('df_es.csv', sep=',', encoding='latin-1', on_bad_lines='skip')

print("Datos cargados correctamente.")
print(f"España: {len(df_es_raw)} filas, {len(df_es_raw.columns)} columnas")
print(f"Alemania: {len(df_de_raw)} filas, {len(df_de_raw.columns)} columnas")

print("\nColumnas España:", df_es_raw.columns.tolist())
print("Columnas Alemania:", df_de_raw.columns.tolist())


Datos cargados correctamente.
España: 790 filas, 9 columnas
Alemania: 46405 filas, 9 columnas

Columnas España: ['brand', 'model', 'price (eur)', 'engine', 'year', 'mileage (kms)', 'fuel', 'gearbox', 'location']
Columnas Alemania: ['mileage', 'make', 'model', 'fuel', 'gear', 'offerType', 'price', 'hp', 'year']


In [ ]:
# Ver nombres actuales de las columnas del CSV de Barcelona
print("Columnas CSV España (Barcelona):")
print(df_es_raw.columns.tolist())

print("\nColumnas CSV Alemania:")
print(df_de_raw.columns.tolist())


Columnas CSV España (Barcelona):
['brand', 'model', 'price (eur)', 'engine', 'year', 'mileage (kms)', 'fuel', 'gearbox', 'location']

Columnas CSV Alemania:
['mileage', 'make', 'model', 'fuel', 'gear', 'offerType', 'price', 'hp', 'year']


2. Unificar columnas

In [ ]:
# España (Barcelona)
df_es = df_es_raw.rename(columns={
    'brand': 'make',
    'price (eur)': 'price',
    'engine': 'hp',                # luego lo limpiamos a número
    'mileage (kms)': 'km',
    'gearbox': 'transmission'
})

# Alemania
df_de = df_de_raw.rename(columns={
    'mileage': 'km',
    'gear': 'transmission'
})

cols_estandar = ['make', 'model', 'price', 'year', 'hp', 'km', 'fuel', 'transmission']

df_es = df_es[cols_estandar].copy()
df_de = df_de[cols_estandar].copy()

print("OK - Columnas finales España:", df_es.columns.tolist())
print("OK - Columnas finales Alemania:", df_de.columns.tolist())


OK - Columnas finales España: ['make', 'model', 'price', 'year', 'hp', 'km', 'fuel', 'transmission']
OK - Columnas finales Alemania: ['make', 'model', 'price', 'year', 'hp', 'km', 'fuel', 'transmission']


3. Limpieza de texto y numeros

In [ ]:
import re

def limpiar_texto(df):
    cols_texto = ['make', 'model', 'fuel', 'transmission']
    for col in cols_texto:
        if col in df.columns:
            df[col] = df[col].astype(str).str.lower().str.strip()
    return df

# Aplicamos la limpieza de texto sbre los datasets
df_es = limpiar_texto(df_es)
df_de = limpiar_texto(df_de)

print("Texto limpio")


Texto limpio (minúsculas)


In [ ]:
import numpy as np

KW_TO_CV = 1.35962

def extraer_cv_desde_engine(s):
    if pd.isna(s):
        return np.nan
    s = str(s).lower()
    
    # Ccoge "230cv" o "230 cv"
    m_cv = re.search(r'(\d{2,3})\s*cv', s)
    if m_cv:
        return float(m_cv.group(1))

    # Coge "169kw" y lo convierte a CV
    m_kw = re.search(r'(\d{2,3})\s*kw', s)
    if m_kw:
        return float(m_kw.group(1)) * KW_TO_CV
    
    return np.nan

# Extraer hp desde df_es_raw (antes de la limpieza numérica)
df_es['hp'] = df_es_raw['engine'].apply(extraer_cv_desde_engine)

# Quitar errores (0 o valores imposibles)
df_es.loc[(df_es['hp'] < 40) | (df_es['hp'] > 700), 'hp'] = np.nan

# Imputación inteligente (por marca-modelo, luego marca, luego global)
hp_mm = df_es.groupby(['make', 'model'])['hp'].transform('median')
hp_m = df_es.groupby('make')['hp'].transform('median')
hp_global = df_es['hp'].median()

df_es['hp'] = df_es['hp'].fillna(hp_mm).fillna(hp_m).fillna(hp_global)

print("HP España extraído:")
print(df_es['hp'].describe())
print(f"NaN restantes: {df_es['hp'].isna().sum()}")


HP España extraído:
count    790.000000
mean     126.098636
std       36.603579
min       60.000000
25%      110.000000
50%      119.646560
75%      135.000000
max      435.000000
Name: hp, dtype: float64
NaN restantes: 0


In [ ]:
def limpiar_numero(valor):
    if pd.isna(valor):
        return None
    texto = str(valor)
    digitos = re.sub(r'\D', '', texto)
    if not digitos:
        return None
    return int(digitos)

# En España limpia: price, year, kmfor col in ['price', 'year', 'km']:
    if col in df_es.columns:
        df_es[col] = df_es[col].apply(limpiar_numero)

# En Alemania limpia:  price, year, km (NO hp, ya está convertido de kW→CV)
for col in ['price', 'year', 'km']:
    if col in df_de.columns:
        df_de[col] = df_de[col].apply(limpiar_numero)


# Dropna (eliminar filas críticas)
df_es = df_es.dropna(subset=['price', 'hp', 'year', 'km'])
df_de = df_de.dropna(subset=['price', 'hp', 'year', 'km'])

print(f"Limpieza completada.")
print(f"España: {len(df_es)} coches")
print(f"   HP España: min={df_es['hp'].min():.0f}, max={df_es['hp'].max():.0f}, mediana={df_es['hp'].median():.0f}")
print(f"Alemania: {len(df_de)} coches")
print(f"   HP Alemania: min={df_de['hp'].min():.0f}, max={df_de['hp'].max():.0f}, mediana={df_de['hp'].median():.0f}")


Limpieza completada.
España: 790 coches
   HP España: min=60, max=435, mediana=120
Alemania: 46376 coches
   HP Alemania: min=1, max=850, mediana=116


In [ ]:

# Conversión kW → CV (en el dataset de Alemania)
KW_TO_CV = 1.35962

df_de['hp'] = df_de['hp'] * KW_TO_CV

print("Potencia Alemania convertida kW → CV")
print(f"   HP Alemania: min={df_de['hp'].min():.1f}, max={df_de['hp'].max():.1f}, mediana={df_de['hp'].median():.1f}")


Potencia Alemania convertida kW → CV
   HP Alemania: min=1.4, max=1155.7, mediana=157.7


4. Imports de Sklearn

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

print("Librerías sklearn cargadas.")


Librerías sklearn cargadas.


5. Entrenamiento del modelo

In [ ]:
# Separar features (X) y target (y) de España
X = df_es.drop('price', axis=1)
y = df_es['price']

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir columnas categóricas y numéricas
categorical_features = ['make', 'model', 'fuel', 'transmission']
numerical_features = ['year', 'hp', 'km']

# Crear pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Entrenar
print("Entrenando modelo...")
model.fit(X_train, y_train)

# Evaluar
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f"Modelo entrenado correctamente.")
print(f"Error Medio Absoluto (MAE): {mae:.2f} €")


Entrenando modelo...
Modelo entrenado correctamente.
Error Medio Absoluto (MAE): 2718.24 €


6. Funcion para los costes de importacion

In [ ]:
def calcular_coste_importacion(row):
    #Calcula costes de traer y matricular un coche.
    precio_compra = row['price']
    year = row['year']
    hp = row['hp']
    
    # Costes fijos
    gastos_fijos = 1150 
    
    # Impuesto matriculación
    antiguedad = 2026 - year 
    
    # Depreciación hacienda
    if antiguedad <= 1: factor = 1.0
    elif antiguedad <= 2: factor = 0.84
    elif antiguedad <= 3: factor = 0.67
    elif antiguedad <= 4: factor = 0.56
    elif antiguedad <= 5: factor = 0.47
    else: factor = 0.30 
    
    valor_nuevo_estimado = hp * 200 
    base_imponible = valor_nuevo_estimado * factor
    
    tipo = 0.0975 if hp > 170 else 0.0475
    
    impuesto = base_imponible * tipo
    return gastos_fijos + impuesto

print("Función de costes definida.")


Función de costes definida.


7. Analisis final

In [ ]:
# Hacemos una copia limpia (no machacamos df_de original)
df_analisis = df_de.copy()

print(f"Dataset Alemania original: {len(df_analisis)} coches")

# Se aplican filtros para que el resultado sea relevante y lo más realista posible
marcas_top = ['audi', 'bmw', 'mercedes-benz', 'volkswagen', 'seat', 'ford', 
              'renault', 'peugeot', 'fiat', 'toyota', 'opel', 'citroen', 'skoda', 'volvo']

df_analisis = df_analisis[df_analisis['make'].isin(marcas_top)]
print(f"Tras filtrar marcas: {len(df_analisis)} coches")

# Quitar precios absurdos
df_analisis = df_analisis[df_analisis['price'] > 2000]

# Rellenar modelos vacíos
df_analisis['model'] = df_analisis['model'].fillna('unknown')

# Filtro potencia (errores)
df_analisis = df_analisis[(df_analisis['hp'] > 50) & (df_analisis['hp'] < 600)]

# FILTRO CLAVE: Quitar microcoches/eléctricos atípicos
modelos_excluir = ['ion', 'twizy', 'zoe', 'c-zero', 'i-miev', 'i3', 'up!', 'mii', 'citigo']
df_analisis = df_analisis[~df_analisis['model'].isin(modelos_excluir)]

print(f"Coches listos para análisis: {len(df_analisis)}")

# 3. PREDICCIÓN (¿Cuánto valdría en España?)
print("Prediciendo precios españoles...")
df_analisis['precio_venta_espana'] = model.predict(df_analisis)

# 4. COSTES (¿Cuánto cuesta traerlo?)
df_analisis['costes_importacion'] = df_analisis.apply(calcular_coste_importacion, axis=1)

# 5. BENEFICIO (La clave del negocio)
df_analisis['beneficio_potencial'] = (
    df_analisis['precio_venta_espana'] - 
    df_analisis['price'] - 
    df_analisis['costes_importacion']
)

# 6. RESULTADOS
top_20 = df_analisis.sort_values('beneficio_potencial', ascending=False).head(20)

print("\n" + "="*70)
print("TOP 20 OPORTUNIDADES DE IMPORTACIÓN")
print("="*70)
cols_mostrar = ['make', 'model', 'year', 'price', 'precio_venta_espana', 'beneficio_potencial']
print(top_20[cols_mostrar].to_string(index=False))

# Guardar CSV final
df_analisis.to_csv('analisis_arbitraje_coches_final.csv', index=False)
print("\nAnálisis guardado en 'analisis_arbitraje_coches_final.csv'")


Dataset Alemania original: 46376 coches
Tras filtrar marcas: 37223 coches
Coches listos para análisis: 35040
Prediciendo precios españoles...

TOP 20 OPORTUNIDADES DE IMPORTACIÓN
         make     model  year  price  precio_venta_espana  beneficio_potencial
         ford     focus  2017   7300              23484.6         13841.533450
          bmw        x5  2011  13900              29962.0         12963.324635
          bmw        x5  2012  14000              29990.5         12891.824635
      renault    kangoo  2020   4676              18710.4         12574.406640
          bmw        x5  2013  16250              31152.9         11804.224635
         opel    zafira  2017   5999              19167.7         10722.234349
      citroen c-elysã©e  2017   8490              20601.9         10512.409628
          bmw        x5  2012  16500              29990.5         10391.824635
      citroen       ds3  2017  10000              21952.6         10376.359130
   volkswagen    passat  2017  